# Use `ibm-watsonx-ai` to deploy a Python script
This notebook contains steps and code to demonstrate how to deploy a Python script with the [ibm-watsonx-ai](https://pypi.python.org/pypi/ibm-watsonx-ai) library available in the PyPI repository. This notebook consists of steps to create a Python script, create a deployment, create and run a job. 

Some familiarity with Python is helpful. This notebook uses Python 3.12.

## Learning goals
The learning goals of this notebook are:

- Create and save a python script.
- Deploy the script using the client library.
- Create and Run a job which utilizes the created deployment.

## Contents
This notebook contains the following parts:

1. [Setup](#section1)
2. [Deployment of Python Script](#section2)
3. [Create and Run Job](#section3)
4. [Clean up](#section4)
5. [Summary](#section5)

<a id='section1'></a>
## 1. Set up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Contact your Cloud Pak for Data administrator and ask them for your account credentials

### Install dependencies
**Note:** `ibm-watsonx-ai` documentation can be found <a href="https://ibm.github.io/watsonx-ai-python-sdk/index.html" target="_blank" rel="noopener no referrer">here</a>.

In [1]:
%pip install -U ibm-watsonx-ai | tail -n 1

#### Define credentials

Authenticate the watsonx.ai Runtime service on IBM Cloud Pak for Data. You need to provide the **admin's** `username` and the platform `url`.

In [ ]:
username = "PASTE YOUR USERNAME HERE"
url = "PASTE THE PLATFORM URL HERE"

Use the **admin's** `api_key` to authenticate watsonx.ai Runtime services:

In [ ]:
import getpass
from ibm_watsonx_ai import Credentials

credentials = Credentials(
    username=username,
    api_key=getpass.getpass("Enter your watsonx.ai API key and hit enter: "),
    url=url,
    instance_id="openshift",
    version="5.2",
)

Alternatively you can use the **admin's** `password`:

In [3]:
import getpass
from ibm_watsonx_ai import Credentials

if "credentials" not in locals() or not credentials.api_key:
    credentials = Credentials(
        username=username,
        password=getpass.getpass("Enter your watsonx.ai password and hit enter: "),
        url=url,
        instance_id="openshift",
        version="5.2",
    )

#### Create `APIClient` instance

In [4]:
from ibm_watsonx_ai import APIClient

client = APIClient(credentials)

### Working with spaces

First of all, you need to create a space that will be used for your work. If you do not have space already created, you can use `{PLATFORM_URL}/ml-runtime/spaces?context=icp4data` to create one.

- Click New Deployment Space
- Create an empty space
- Go to space `Settings` tab
- Copy `space_id` and paste it below

**Tip**: You can also use SDK to prepare the space for your work. More information can be found [here](https://github.com/IBM/watsonx-ai-samples/blob/master/cpd5.2/notebooks/python_sdk/instance-management/Space%20management.ipynb).

**Action**: Assign space ID below

In [ ]:
space_id = "PASTE YOUR SPACE ID HERE"

You can use `list` method to print all existing spaces.

In [ ]:
client.spaces.list(limit=10)

To be able to interact with all resources available in Watson Machine Learning, you need to set **space** which you will be using.

In [6]:
client.set.default_space(space_id)

'SUCCESS'

### Software specification

You can use popular tools, libraries, and frameworks to train and deploy machine learning models and functions.

The following list shows the predefined (base) model types and software specifications.

In [7]:
client.software_specifications.list()

,NAME,ID,TYPE,STATE,REPLACEMENT
0,autoai-kb_rt25.1-py3.12,046f5252-0d07-5c23-81ef-18a1a31cdce3,base,supported,
1,watsonx-cfm-caikit-1.0,0cee3c55-472f-57b1-84bd-72f5d066dbe4,base,supported,
2,watsonx-textgen-fm-1.0,129aec82-7e65-5c78-b812-4c0a74b916f5,base,supported,
3,masking-flows-spark,13666829-5570-53a7-927b-52d42a101d93,base,supported,
4,autoai-ts_rt25.1-py3.12,1c0c9e1d-8523-520f-867e-102173f16f5e,base,supported,
5,tensorflow_rt24.1-py3.11,2c33167d-b11c-5490-a305-3e5e95db5c4d,base,supported,
6,pytorch-onnx_rt24.1-py3.11,2da185aa-eac3-59a5-bb8e-0e5b60458a15,base,supported,
7,tensorflow_rt24.1-py3.11-edt,2f53b747-c01a-58a6-a20c-8390ecfd8625,base,supported,
8,kernel-spark3.5-py3.11,30ad99e2-7cfb-511a-9a3e-4cc308071fd3,base,supported,
9,kernel-spark3.5-r4.3,3668b4d5-b8b8-5926-badf-1134137613d4,base,supported,


You can select the software specification using the function below.

In [8]:
base_sw_spec_id = client.software_specifications.get_uid_by_name("runtime-25.1-py3.12")
print(base_sw_spec_id)

f47ae1c3-198e-5718-b59d-2ea471561e9e


<a id='section2'></a>
## 2. Python Script Deployment

### Save Python Script

This file will be saved locally so you can deploy and run it later.

In [9]:
%%writefile /tmp/CreateSwSpec_script.py

import sys
import time
import os

outfname = os.path.join(os.environ.get("BATCH_OUTPUT_DIR"), "swspec.log")
with open(outfname, "w") as file:
    try:
        file.write("The deployed python script was run successfully!")
    except Exception as exc:
        file.write(f"The deployed python script failed: {exc}")
        file.write("sys path:")
        file.write(sys.path)

Writing /tmp/CreateSwSpec_script.py


The file should be successfully created. 
To check its content, you can use the command below.

In [10]:
%cat /tmp/CreateSwSpec_script.py


import sys
import time
import os

outfname = os.path.join(os.environ.get("BATCH_OUTPUT_DIR"), "swspec.log")
with open(outfname, "w") as file:
    try:
        file.write("The deployed python script was run successfully!")
    except Exception as exc:
        file.write(f"The deployed python script failed: {exc}")
        file.write("sys path:")
        file.write(sys.path)


### Deployment of Python Script

You can store and deploy a Python script and get its details by running the code in following cells. 

In [11]:
meta_props = {
    client.script.ConfigurationMetaNames.NAME: "Python script",
    client.script.ConfigurationMetaNames.SOFTWARE_SPEC_UID: base_sw_spec_id,
}

In [12]:
script_path = "/tmp/CreateSwSpec_script.py"
script_details = client.script.store(meta_props, file_path=script_path)
script_id = client.script.get_id(script_details)
print("Created script ", script_id)

Creating Script asset...
SUCCESS
Created script  91b49ece-f724-4cc8-b567-29a6638a222e


In [13]:
script_details

{'metadata': {'name': 'Python script',
  'guid': '91b49ece-f724-4cc8-b567-29a6638a222e',
  'href': '/v2/assets/91b49ece-f724-4cc8-b567-29a6638a222e?space_id=8a13841b-df99-4b4d-bf2a-161ad2e33980',
  'asset_type': 'script',
  'created_at': '2025-05-13T12:38:10Z',
  'last_updated_at': '2025-05-13T12:38:10Z',
  'space_id': '8a13841b-df99-4b4d-bf2a-161ad2e33980',
  'description': ''},
 'entity': {'script': {'language': {'name': 'python3'},
   'software_spec': {'base_id': 'f47ae1c3-198e-5718-b59d-2ea471561e9e'}}}}

In [14]:
deployment_meta_props = {
    client.deployments.ConfigurationMetaNames.NAME: "pyscript_deployment",
    client.deployments.ConfigurationMetaNames.BATCH: {},
    client.deployments.ConfigurationMetaNames.HARDWARE_SPEC: {"name": "S"},
}

deployment_details = client.deployments.create(script_id, deployment_meta_props)
deployment_id = client.deployments.get_id(deployment_details)



######################################################################################

Synchronous deployment creation for id: '91b49ece-f724-4cc8-b567-29a6638a222e' started

######################################################################################


ready.


-----------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_id='c1ae1cce-8dc7-4862-9c69-1de9358f81bf'
-----------------------------------------------------------------------------------------------




In [15]:
deployment_details

{'entity': {'asset': {'id': '91b49ece-f724-4cc8-b567-29a6638a222e'},
  'batch': {},
  'chat_enabled': False,
  'custom': {},
  'deployed_asset_type': 'py_script',
  'hardware_spec': {'id': 'e7ed1d6c-2e89-42d7-aed5-863b972c1d2b',
   'name': 'S',
   'num_nodes': 1},
  'name': 'pyscript_deployment',
  'space_id': '8a13841b-df99-4b4d-bf2a-161ad2e33980',
  'status': {'state': 'ready'}},
 'metadata': {'created_at': '2025-05-13T12:38:31.241Z',
  'id': 'c1ae1cce-8dc7-4862-9c69-1de9358f81bf',
  'modified_at': '2025-05-13T12:38:31.241Z',
  'name': 'pyscript_deployment',
  'owner': '1000331001',
  'space_id': '8a13841b-df99-4b4d-bf2a-161ad2e33980'}}

<a id='section3'></a>
## 3. Create and Run job

Run the following cells to create and run a job with the deployed script.

In [ ]:
import time


def poll_async_job(wml_client, job_uid):
    while True:
        job_status = wml_client.deployments.get_job_status(job_uid)
        print(job_status)
        state = job_status["state"]
        if state == "completed" or "fail" in state:
            return wml_client.deployments.get_job_details(job_uid)
        time.sleep(5)

In [17]:
job_payload_ref = {
    client.deployments.ScoringMetaNames.OUTPUT_DATA_REFERENCE: {
        "type": "data_asset",
        "location": {"name": "deploy_test_script-out"},
    }
}

job = client.deployments.create_job(deployment_id, meta_props=job_payload_ref)
job_id = client.deployments.get_job_id(job)

In [18]:
job_details = poll_async_job(client, job_id)

{'completed_at': '', 'running_at': '', 'state': 'queued'}
{'completed_at': '', 'running_at': '', 'state': 'queued'}
{'completed_at': '', 'running_at': '', 'state': 'queued'}
{'completed_at': '', 'running_at': '', 'state': 'queued'}
{'completed_at': '', 'running_at': '', 'state': 'queued'}
{'completed_at': '', 'running_at': '2025-05-13T12:39:14.200Z', 'state': 'running'}
{'completed_at': '2025-05-13T12:39:18.926414Z', 'running_at': '2025-05-13T12:39:14.184392Z', 'state': 'completed'}


In [19]:
client.data_assets.list()

,NAME,ASSET_TYPE,SIZE,ASSET_ID
0,deploy_test_script-out,data_asset,166,07ced9b8-c914-4d81-8dc9-fdffabcc07bb


To download the asset run the code below. It will be downloaded as a zip archive.

In [ ]:
scoring_params = client.deployments.get_job_details(job_id)

from ibm_watsonx_ai.helpers import DataConnection

output_data_connection = DataConnection.from_dict(
    scoring_params["entity"]["scoring"]["output_data_reference"]
)
output_data_connection.set_client(client)

output_data_connection.download("/tmp/CreateSwSpec_script_result.zip")

In [21]:
import zipfile

with zipfile.ZipFile("/tmp/CreateSwSpec_script_result.zip") as zip_file:
    with zip_file.open("swspec.log") as file:
        print(file.read().decode())

The deployed python script was run successfully!


<a id='section4'></a>
## 4. Cleanup 

If you want to clean up all created assets:
- experiments
- trainings
- pipelines
- model definitions
- models
- functions
- deployments

please follow up this sample [notebook](https://github.com/IBM/watsonx-ai-samples/blob/master/cpd5.2/notebooks/python_sdk/instance-management/Machine%20Learning%20artifacts%20management.ipynb).

<a id='section5'></a>
## 5. Summary

You successfully completed this notebook! You learned how to create and deploy a python script, and create and run a job using Watson Machine Learning.

Check out our _<a href="https://ibm.github.io/watsonx-ai-python-sdk/samples.html" target="_blank" rel="noopener no referrer">Online Documentation</a>_ for more samples, tutorials, documentation, how-tos, and blog posts. 


### Author

**Jakub Żywiecki**, Software Engineer

Copyright © 2024-2025 IBM. This notebook and its source code are released under the terms of the MIT License.